This is a parser to get all annotations in a single file for kinfin analysis. Pull in files from the following two folders:

/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/enrichment_analysis/pa_26062017
and make a effector tablist for effectors into the same folder from

/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/enrichment_analysis/lists

The layout of functional annotation file is as follows.

 \#protein_id GO IPR SignalP_EUK Pfam Effector Merops KEGG

with the following characteriztics

protein_id is simple the id used in clustering as well
GO is a list of GO terms as GO:XX;GO:XX
IPR is the ; sperated list of IPRterm:count for now this count will be one in all cases
The same is true for all other categories as well.

In [2]:
import pandas as pd
import os
import re
from Bio import SeqIO
import pysam
from Bio.SeqRecord import SeqRecord
from pybedtools import BedTool
import numpy as np
import pybedtools
import time
import sys
import matplotlib.pyplot as plt
import subprocess
import shutil
from collections import Counter

In [3]:
genome = 'Pst_104E_v12_p_ctg'
LIST_FOLDER = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/enrichment_analysis/lists'
ANNOTATION_FOLDER = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/enrichment_analysis/pa_26062017'
BASE_FOLDER = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12'
ASSEMBLY_FOLDER = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/032017_assembly'
KINFIN_FOLDER = os.path.join(BASE_FOLDER, 'KinFin')
if not os.path.exists(KINFIN_FOLDER):
    os.mkdir(KINFIN_FOLDER)
#out file name for kinfin
kinfin_out_fn = os.path.join(KINFIN_FOLDER, genome + '.functional_annotation.txt')

In [4]:
#make an initial dataframe that has the protein_id has index
protein_fn = [os.path.join(ASSEMBLY_FOLDER, x) for x in os.listdir(ASSEMBLY_FOLDER) if genome in x\
             and  x.endswith('protein.fa')][0]

In [5]:
#now generate a list of ids and length
protein_id_list = []
protein_len_list = []
for seq in SeqIO.parse(protein_fn, 'fasta'):
    protein_id_list.append(seq.id)
    protein_len_list.append(len(seq.seq))
#make a dataframe out of it
kinfin_fa_df = pd.concat([pd.Series(protein_id_list, name='#protein_id'),\
                          pd.Series(protein_len_list, name='protein_len')], axis=1)

In [6]:
#get the GO list and add to the dataframe
GO_fn = [os.path.join(ANNOTATION_FOLDER, x) for x in os.listdir(ANNOTATION_FOLDER) \
        if x.startswith(genome) and x.endswith('GO_combined.tablist')][0]
GO_df = pd.read_csv(GO_fn, header=None, sep='\t', names=['#protein_id', 'GO'])
#merge the GO_df and the kinfin_df
kinfin_fa_df = pd.merge(kinfin_fa_df, GO_df, how='outer', on='#protein_id')

In [7]:
#now pull in all the annotations list file names into a dictionary
anno_keys = ['merops', 'Pfam', 'busco', 'iprscan', 'SignalP_EUK', 'dbCAN', 'KEGG_combined', 'OG']
anno_fn_dict = {}
for key in anno_keys:
    anno_fn_dict[key] = [os.path.join(ANNOTATION_FOLDER, x) for x in os.listdir(ANNOTATION_FOLDER)\
                        if x.startswith(genome) and key in x][0]

In [8]:
def kf_count(_comma_string):
    """
    Quick function that confered a tablist ; seperated id list to a kinfin suitable occurance count.
    """
    _list = _comma_string.split(';')
    _dict = Counter(_list)
    _new_string = ''
    for x in _dict.keys():
        _new_string = '%s;%s:%i'% (_new_string,x, _dict[x])
    return _new_string[1:]

In [9]:
#now loop throught those anno_fn_dict and pull in the dataframes
kinfin_fa_df
for key in anno_keys:
    #if key in ['merops', ]
    column_id = key.upper()
    tmp_df = pd.read_csv(anno_fn_dict[key], sep='\t', header=None, names=['#protein_id', key])
    #remove the ":" which a left over from funnanotate
    if ":" in tmp_df.loc[0,key]:
        tmp_df[column_id] = tmp_df[key].apply(lambda x: x[x.rindex(':')+1:])
    else:
        tmp_df[column_id] = tmp_df[key]
    #transform the tablist to a counter list sperated by ':'
    tmp_df[column_id] = tmp_df[column_id].apply(lambda x: kf_count(x))
    kinfin_fa_df = pd.merge(kinfin_fa_df, tmp_df.loc[:, ['#protein_id', column_id]], how='outer', on="#protein_id")


This fixes it for now. Now pull in the effector list and the haustoria expression list as well.

In [10]:
#now get haustoria expressed genes and effector genes (effectorP and in planta upregulated secreted genes)
#for the primary contig cluster 15 is haustoria expressed in for the haplogtigs cluster 15
p_effectorP_fn = os.path.join(LIST_FOLDER, 'Pst_104E_v12_p_effector.list')
p_haustoria_fn = os.path.join(LIST_FOLDER, 'Pst_104E_v12_cluster_8.list')
#now add the those to the dataframe
p_effectorP_df = pd.read_csv(p_effectorP_fn, sep='\t', header=None, names=['#protein_id'])
p_effectorP_df['EFFECTOR'] = 'Effector:1'
kinfin_fa_df = pd.merge(kinfin_fa_df, p_effectorP_df, how='outer', on="#protein_id")
#now add the haustoria expression
p_haustoria_df = pd.read_csv(p_haustoria_fn, sep='\t', header=None, names=['#protein_id'])
p_haustoria_df["SHAUSTORIA"] = 'Haustoria:1'
kinfin_fa_df = pd.merge(kinfin_fa_df, p_haustoria_df, how='outer', on="#protein_id")

In [11]:
#now write out two versions one with length included and one without
kinfin_fa_df.fillna('None', inplace=True)

In [19]:
kinfin_out_fn

'/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/KinFin/Pst_104E_v12_p_ctg.functional_annotation.txt'

In [24]:
kinfin_long_fn = kinfin_out_fn.replace('.txt', '_wlength.txt')
kinfin_fa_df.to_csv(kinfin_long_fn, sep='\t', index=None)
#now write out kinfin version
kinfin_fa_df.drop('protein_len', 1).to_csv(kinfin_out_fn, sep='\t', index=None)

In [23]:
kinfin_fa_df.drop('protein_len', 1).head()

,#protein_id,GO,MEROPS,PFAM,BUSCO,IPRSCAN,SIGNALP_EUK,DBCAN,KEGG_COMBINED,OG,EFFECTOR,SHAUSTORIA
0,evm.model.pcontig_041.101,None,None,None,None,None,None,None,None,0ZGCE@NOG:1;0JJMJ@euNOG:1;14B0R@opiNOG:1;095NX...,None,None
1,evm.model.pcontig_018.256,GO:0000271;GO:0003674;GO:0003824;GO:0004610;GO...,None,PF00408:1;PF02878:1,EOG092R06LD:1,IPR005843:3;IPR005844:1;IPR016657:2;IPR016066:...,None,None,map01110:1;map00520:1,COG1109@NOG:1;KOG2537@euNOG:1;0PHNA@fuNOG:1;09...,None,None
2,evm.model.pcontig_018.216,GO:0000003;GO:0003006;GO:0005575;GO:0005622;GO...,None,PF03635:1,EOG092R022P:1,IPR005378:5,None,None,None,0XNXC@NOG:1;0PGCB@fuNOG:1;12NYE@opiNOG:1;KOG11...,None,None
3,evm.model.pcontig_014.344,GO:0004190;GO:0006508,None,PF00077:1,None,IPR001969:1;IPR021109:2;IPR001995:1;IPR018061:1,None,None,None,COG2801@NOG:1;KOG0017@euNOG:1;13IVJ@opiNOG:1;0...,None,None
4,evm.model.pcontig_022.278,GO:0000086;GO:0000166;GO:0000278;GO:0000322;GO...,None,PF00069:1;PF16579:1,None,IPR032270:1;IPR011009:1;IPR000719:3;IPR008271:...,None,None,map04151:1;map04150:1;map04113:1,0PGMD@fuNOG:1;0927K@basNOG:1;KOG0586@euNOG:1;0...,None,None


In [ ]:
test

In [ ]:
ext_df.head()

In [ ]:
test[test.rindex(':')+1:]

In [ ]:
kinfin_fa_df.head()

In [ ]:
#pull out all proteins that are in the final assembly
p_protein_list = []
protein_fa_file = [x for x in os.listdir(BASE_A_PATH) if p_genome in x and x.endswith('anno.protein.fa')][0]
for protein in SeqIO.parse(os.path.join(BASE_A_PATH, protein_fa_file) , 'fasta'):
    p_protein_list.append(protein.id)

In [ ]:
eggnog_blast_header = 'query_name seed_eggNOG_ortholog seed_ortholog_evalue seed_ortholog_score predicted_gene_name \
GO_terms KEGG_pathways Annotation_tax_scope OGs bestOG|evalue|score COG cat eggNOG annot'.split(' ')

In [ ]:
eggnog_blast_header

In [ ]:
eggnog_blast_df = pd.read_csv(os.path.join(BASE_FOLDER, EGGNOG_BLAST_FILE), sep ='\t', header=None, names=eggnog_blast_header, skiprows=3)

In [ ]:
eggnog_blast_df.iloc[1,:]

In [ ]:
eggnog_blast_df.fillna(0, inplace =True)

In [ ]:
eggnog_blast_df.columns

In [ ]:
DBs

In [ ]:
#pick all annotation columns. One at a time and write them out as tab file or annotations files. The later can be used
#to annotate gff files using gag.py
DBs = [x for x in eggnog_blast_df.columns.tolist()[4:] if x not in ['GO_terms','KEGG_pathways', 'OGs' ] ]
eggnog_blast_df['note'] = 'note'
for db in DBs:
    if len(eggnog_blast_df[eggnog_blast_df[db] !=0]) > 0:
        eggnog_blast_df[eggnog_blast_df[db] !=0].loc[:,['query_name',db]]\
        .to_csv(os.path.join(OUT_PATH, db+'_terms.tab'), sep='\t', header =None, index = None)
        eggnog_blast_df[eggnog_blast_df[db] !=0].loc[:,['query_name', 'note', db]]\
        .to_csv(os.path.join(OUT_PATH, 'annotations.' +db+'.txt'), sep='\t', header =None, index = None)
    
    

In [ ]:
working_dict = {}
eggnog_blast_df['GO_terms'] = eggnog_blast_df.GO_terms.str.split(',')

eggnog_blast_df[eggnog_blast_df['GO_terms'] != 'NaN']

In [ ]:
eggnog_blast_df['GO_terms']

In [ ]:
#process the pathway files
interpro_by_protein_KEGG = interpro_df[interpro_df.Pathway_IDs.str.contains('KEGG' or 'MetaCyc' or 'Reactome').fillna(False)]

interpro_by_protein_KEGG = interpro_by_protein_KEGG.groupby('Updated_Protein_ID')

#pull out all the KEGG terms and write them out as annotation files 
interpro_by_protein_KEGG = interpro_by_protein.Pathway_IDs.apply(set)

#remove everything without KEGG term attached
interpro_by_protein_KEGG = interpro_by_protein_KEGG[~(interpro_by_protein_KEGG == {0})]

interpro_by_protein_KEGG_dict = dict(zip(interpro_by_protein_KEGG.index, interpro_by_protein_KEGG))

ALL_KEGG_LIST = []
ALL_PROTEIN_INDEX_LIST = []
for key in list(interpro_by_protein_KEGG_dict.keys()):
    _tmp_list = list(interpro_by_protein_KEGG_dict[key])
    #remove 0 
    _tmp_list = [x for x in _tmp_list if x != 0]
    new_value = []
    for x in _tmp_list: 
        if '|' in x:
            _list = x.split('|')
            for y in _list:
                new_value.append(y)
        else:
            new_value.append(x)
    new_value = list(set(new_value))
    new_index = [key]*len(new_value)
    ALL_KEGG_LIST += new_value
    ALL_PROTEIN_INDEX_LIST += new_index


KEGG_df = pd.DataFrame([ALL_PROTEIN_INDEX_LIST, ALL_KEGG_LIST]).T
KEGG_df.rename(columns={0:'Updated_Protein_ID', 1:'DB_ID'}, inplace=True)

KEGG_df.to_csv(os.path.join(OUT_PATH_P_ALL, 'Pathway_terms_ipr_all.tab') , sep = '\t', header =None, index=None)
KEGG_df[KEGG_df.Updated_Protein_ID.isin(p_protein_list)].to_csv(os.path.join(OUT_PATH_P, 'Pathway_terms_ipr.tab') , sep = '\t', header =None, index=None)
#write out annotations
KEGG_df['Transfer_ID'] = 'note'
KEGG_df.loc[:,['Updated_Protein_ID', 'Transfer_ID','DB_ID']].to_csv(os.path.join(OUT_PATH_P_ALL, 'annotations.Pathway_all.txt') , sep = '\t', header =None, index=None)
KEGG_df[KEGG_df.Updated_Protein_ID.isin(p_protein_list)].loc[:,['Updated_Protein_ID', 'Transfer_ID','DB_ID']].to_csv(os.path.join(OUT_PATH_P, 'annotations.Pathway.txt') , sep = '\t', header =None, index=None)

In [ ]:
#pull out all the GO terms and write them out as annotation files 
interpro_by_protein_GO = interpro_by_protein.GO_terms.apply(set)

#remove everything without GO term attached
interpro_by_protein_GO = interpro_by_protein_GO[~(interpro_by_protein_GO == {0})]

interpro_by_protein_GO_dict = dict(zip(interpro_by_protein_GO.index, interpro_by_protein_GO))

ALL_GO_LIST = []
ALL_PROTEIN_INDEX_LIST = []
for key in list(interpro_by_protein_GO_dict.keys()):
    _tmp_list = list(interpro_by_protein_GO_dict[key])
    #remove 0 
    _tmp_list = [x for x in _tmp_list if x != 0]
    new_value = []
    for x in _tmp_list: 
        if '|' in x:
            _list = x.split('|')
            for y in _list:
                new_value.append(y)
        else:
            new_value.append(x)
    new_value = list(set(new_value))
    new_index = [key]*len(new_value)
    ALL_GO_LIST += new_value
    ALL_PROTEIN_INDEX_LIST += new_index


GO_df = pd.DataFrame([ALL_PROTEIN_INDEX_LIST, ALL_GO_LIST]).T
GO_df.rename(columns={0:'Updated_Protein_ID', 1:'DB_ID'}, inplace=True)

GO_df.to_csv(os.path.join(OUT_PATH_P_ALL, 'GO_terms_ipr_all.tab') , sep = '\t', header =None, index=None)
GO_df[GO_df.Updated_Protein_ID.isin(p_protein_list)].to_csv(os.path.join(OUT_PATH_P, 'GO_terms_ipr.tab') , sep = '\t', header =None, index=None)
#write out annotations
GO_df['Transfer_ID'] = 'note'
GO_df.loc[:,['Updated_Protein_ID', 'Transfer_ID','DB_ID']].to_csv(os.path.join(OUT_PATH_P_ALL, 'annotations.GO_all.txt') , sep = '\t', header =None, index=None)
GO_df[GO_df.Updated_Protein_ID.isin(p_protein_list)].loc[:,['Updated_Protein_ID', 'Transfer_ID','DB_ID']].to_csv(os.path.join(OUT_PATH_P, 'annotations.GO.txt') , sep = '\t', header =None, index=None)

In [ ]:
#filter out other annotations files including busco, dbCAN, merops, swissprot
annotation_files = [os.path.join(ANNOTATION_FILE_BASE_PATH, x) for x in os.listdir(ANNOTATION_FILE_BASE_PATH) \
                    if x.startswith('annotations') and (x.endswith('busco.txt') or x.endswith('dbCAN.txt') or x.endswith('merops.txt') or x.endswith('swissprot.txt'))]

In [ ]:
for anno in annotation_files:
    _tmp_df = pd.read_csv(anno, header=None, sep='\t', names=['Protein_ID','Transfer_ID', 'DB_ID'])
    anno_file_name = anno.split('/')[-1]
    anno_midfix = anno_file_name.split('.')[1]
    _tmp_df['Updated_Protein_ID'] = protein_id_conversion(_tmp_df)
    #write out filtered down tab file
    _tmp_df[(_tmp_df.Updated_Protein_ID.isin(p_protein_list))].loc[:,['Updated_Protein_ID', 'DB_ID']].to_csv(os.path.join(OUT_PATH_P, anno_midfix+'_terms.tab'), sep = '\t', header =None, index=None)
    #write out the filtered annotation file
    _tmp_df[(_tmp_df.Updated_Protein_ID.isin(p_protein_list))].loc[:,['Updated_Protein_ID', 'Transfer_ID','DB_ID']].\
    to_csv(os.path.join(OUT_PATH_P, anno_file_name ), sep = '\t', header =None, index=None)
     #write out not down tab file
    _tmp_df.loc[:,['Updated_Protein_ID', 'DB_ID']].to_csv(os.path.join(OUT_PATH_P_ALL, anno_midfix+'_terms_all.tab' ), sep = '\t', header =None, index=None)
    #write out the filtered annotation file
    _tmp_df.loc[:,['Updated_Protein_ID', 'Transfer_ID','DB_ID']].to_csv(os.path.join(OUT_PATH_P_ALL, anno_file_name.replace('.txt', '_all.txt') ), sep = '\t', header =None, index=None)